
# pandas: Zero to Hero 🐼

Welcome! This notebook teaches **pandas** from the ground up—fully self-paced.
You'll learn Series and DataFrame fundamentals, indexing, reshaping, joins,
groupby, time series, text handling, and performance tips—plus a mini-project.

**You will:**
- Understand **Series** and **DataFrame**, indices, and dtypes
- Load & save data (`read_csv`, `to_csv`, etc.)
- Select, filter, and transform data with `loc`, `iloc`, boolean masks, `assign`
- Handle missing data (`isna`, `fillna`, `dropna`)
- Aggregate with `groupby`, build pivot tables, reshape with `melt`/`pivot`
- Merge/join/concat datasets
- Work with text (`.str`) and dates (`to_datetime`, resampling, rolling windows)
- Use categorical dtype for memory/speed
- Apply functions (`apply`, `map`, `transform`) and vectorization
- Plot with Matplotlib (one chart per figure, no explicit colors)
- Practice with exercises and a capstone mini-project


## 0) Setup

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# For reproducibility
rng = np.random.default_rng(42)

pd.__version__



## 1) Core Concepts: Series and DataFrame

- **Series**: 1D labeled array (values + index)
- **DataFrame**: 2D table of columns (each column is a Series)
- **Index**: labels for rows (and optionally columns for DataFrame)
- **Dtype**: data type per Series/column (e.g., `int64`, `float64`, `object`, `category`)


In [ ]:

s = pd.Series([10, 20, 30], index=["a", "b", "c"], name="scores")
print("Series:\n", s)
print("Values:", s.values, "Index:", s.index, "Dtype:", s.dtype)

df = pd.DataFrame({
    "size": [650, 800, 1200, 1500],
    "bedrooms": [1, 2, 3, 3],
    "price": [120000, 155000, 240000, 310000]
}, index=["h1", "h2", "h3", "h4"])
print("\nDataFrame:\n", df)
print("Shape:", df.shape, "Index:", df.index, "Columns:", df.columns)



## 2) Creating DataFrames

Common ways:
- From dict of lists/arrays
- From list of dicts
- From NumPy arrays with column names


In [ ]:

# Dict of lists
df1 = pd.DataFrame({"A":[1,2,3], "B":[4,5,6]})
# List of dicts
df2 = pd.DataFrame([{"A":1,"B":4},{"A":2,"B":5},{"A":3,"B":6}])
# NumPy arrays
arr = np.arange(12).reshape(4,3)
df3 = pd.DataFrame(arr, columns=["x","y","z"])

print(df1, "\n")
print(df2, "\n")
print(df3)



## 3) Reading and Writing Data

- `pd.read_csv("file.csv")`
- `df.to_csv("file.csv", index=False)`

Useful `read_csv` args:
- `sep`, `header`, `names`, `usecols`
- `dtype`, `na_values`
- `parse_dates`, `dayfirst`
- `nrows`, `skiprows`, `encoding`


In [ ]:

from io import StringIO

csv = StringIO("""city,month,temp,visitors
Alpha,2025-01,5,120
Alpha,2025-02,8,160
Alpha,2025-03,12,220
Beta,2025-01,-2,80
Beta,2025-02,1,110
Beta,2025-03,6,150
Gamma,2025-01,10,200
Gamma,2025-02,12,230
Gamma,2025-03,15,300
""")
df_city = pd.read_csv(csv, parse_dates=["month"])
print(df_city.head())
# Save example
path = "/mnt/data/cities.csv"
df_city.to_csv(path, index=False)
print("Saved to:", path)



## 4) Inspecting Data

- `df.head()`, `df.tail()`
- `df.info()` (dtypes + nulls)
- `df.describe()` (numeric summary)
- `df.sample(n)` random rows
- `df.isna().sum()` missing counts


In [ ]:

print(df_city.head(3))
print(df_city.info())
print(df_city.describe(numeric_only=True))
print("Missing per column:\n", df_city.isna().sum())
print("Random sample:\n", df_city.sample(3, random_state=0))



## 5) Selecting & Indexing: `loc` and `iloc`

- `df["col"]` / `df[["c1","c2"]]` select columns
- `df.loc[row_label, col_label]` label-based
- `df.iloc[row_idx, col_idx]` position-based

**Tip:** prefer `loc`/`iloc` for clarity.


In [ ]:

print(df_city[["city","temp"]].head())

# Set an index for demonstration (we'll reset later)
df_idx = df_city.set_index("city")
print("\nBy label with loc (rows with label 'Alpha'):\n", df_idx.loc["Alpha"])

print("\nPosition with iloc (first 2 rows, first 2 cols):\n", df_city.iloc[:2, :2])

# Reset index back
df_idx = df_idx.reset_index()



## 6) Filtering Rows: Boolean Masks & `query`

- Boolean masks: `df[df["col"] > 10]`
- `query`: string expression, great for multiple conditions


In [ ]:

# Boolean mask
hot = df_city[df_city["temp"] >= 10]

# Multiple conditions (& and |) need parentheses
alpha_hot = df_city[(df_city["city"]=="Alpha") & (df_city["temp"]>=10)]

# query
beta_cool = df_city.query('city == "Beta" and temp <= 1')

print("hot:\n", hot)
print("\nalpha_hot:\n", alpha_hot)
print("\nbeta_cool:\n", beta_cool)



## 7) Creating/Transforming Columns: `assign`, vectorized ops

- Vectorized math: `df["new"] = df["a"] + df["b"]`
- `assign(...)` returns a new DataFrame (method-chaining friendly)
- Avoid chained indexing for setting; prefer `.loc` or `.assign`


In [ ]:

df_aug = (df_city
          .assign(temp_F = df_city["temp"] * 9/5 + 32,
                  is_peak = df_city["visitors"] > 200))

print(df_aug.head())

# Safe in-place assignment using .loc
df_city.loc[df_city["city"]=="Beta", "visitors"] += 5
print("\nAfter in-place increment for Beta visitors:\n", df_city.head())



## 8) Missing Data: `isna`, `fillna`, `dropna`

- Detect: `isna()` / `notna()`
- Fill: `fillna(value)`
- Drop: `dropna()` (rows with any nulls, configurable)


In [ ]:

df_missing = df_city.copy()
df_missing.loc[df_missing.index[1], "visitors"] = np.nan
df_missing.loc[df_missing.index[4], "temp"] = np.nan

print("Missing counts:\n", df_missing.isna().sum())

# Fill numeric with mean, categorical with a sentinel
filled = df_missing.copy()
filled["visitors"] = filled["visitors"].fillna(filled["visitors"].mean())
filled["temp"] = filled["temp"].fillna(filled["temp"].median())
print("\nAfter fillna (head):\n", filled.head())

# Drop rows with any NaN
dropped = df_missing.dropna()
print("\nAfter dropna (rows kept):", len(dropped))



## 9) Sorting & Duplicates

- Sort rows: `sort_values(by=[...], ascending=...)`
- Sort columns: `sort_index(axis=1)`
- Remove duplicate rows: `drop_duplicates(subset=[...], keep="first")`


In [ ]:

sorted_df = df_city.sort_values(by=["city","temp"], ascending=[True, False])
print(sorted_df.head())

dupes = pd.DataFrame({"A":[1,1,2,2,3], "B":[9,9,8,7,7]})
print("\nOriginal with dupes:\n", dupes)
print("Drop duplicate rows:\n", dupes.drop_duplicates())

print("\nSort columns by name:\n", df_city.sort_index(axis=1).head())



## 10) GroupBy: Split–Apply–Combine

- `groupby` groups rows by keys (e.g., a column like `city`)
- `agg` computes aggregations per group (sum, mean, count, ...)
- `transform` returns aligned result (same shape as original, for feature engineering)
- `apply` for custom python functions (use sparingly for performance)


In [ ]:

g = df_city.groupby("city")

print("Mean by city:\n", g[["temp","visitors"]].mean())

# transform example: z-score of visitors within each city
df_city["visitors_z"] = g["visitors"].transform(lambda x: (x - x.mean())/x.std())
print("\nWith visitors_z:\n", df_city.head())

# multi-agg
agg_df = g.agg(visitors_mean=("visitors","mean"),
               temp_max=("temp","max"),
               n=("visitors","count"))
print("\nMulti-agg:\n", agg_df)



## 11) Pivot Tables

`pivot_table` aggregates values across two dimensions (index & columns).


In [ ]:

pivot = pd.pivot_table(df_city,
                       values="visitors",
                       index="city",
                       columns=df_city["month"].dt.month,
                       aggfunc="mean",
                       fill_value=0)
print(pivot)



## 12) Reshaping: `melt` and `pivot`

- `melt`: wide → long (unpivot)
- `pivot`: long → wide (inverse of melt if keys unique)


In [ ]:

wide = pd.DataFrame({
    "city": ["Alpha","Beta"],
    "Jan": [120,80],
    "Feb": [160,110],
    "Mar": [220,150]
})

long = wide.melt(id_vars="city", var_name="month", value_name="visitors")
print("Long format:\n", long)

wide_back = long.pivot(index="city", columns="month", values="visitors")
print("\nWide back:\n", wide_back)



## 13) Merging, Joining, Concatenating

- `pd.merge(left, right, on="key", how="inner/left/right/outer")`
- `df.join(other, on=..., how=..., lsuffix=..., rsuffix=...)` (index-based)
- `pd.concat([df1, df2], axis=0/1)`


In [ ]:

cities = pd.DataFrame({"city":["Alpha","Beta","Gamma"], "region":["North","North","South"]})
avg_income = pd.DataFrame({"city":["Alpha","Beta","Gamma"], "income":[50_000, 42_000, 55_000]})

merged = pd.merge(df_city, cities, on="city", how="left")
merged = pd.merge(merged, avg_income, on="city", how="left")
print(merged.head())

# Concatenate rows
df_extra = merged.iloc[:2].copy()
concat_rows = pd.concat([merged, df_extra], axis=0, ignore_index=True)
print("\nConcatenated rows (tail):\n", concat_rows.tail())



## 14) String Methods (`.str`)

Vectorized string ops for Series of dtype `object` or `string`:
- case changes: `.str.lower()`, `.str.upper()`, `.str.title()`
- contains/split/extract/replace: `.str.contains`, `.str.split`, `.str.extract`, `.str.replace`


In [ ]:

cities_series = pd.Series([" New York ", "london", "Tehran", "san-francisco"])
clean = (cities_series.str.strip()
         .str.replace("-", " ", regex=False)
         .str.title())
print(clean)

# contains & extract
s2 = pd.Series(["ID:123", "ID:999", "NONE"])
mask = s2.str.contains(r"ID:\d+", regex=True)
ids = s2.str.extract(r"(\d+)", expand=False)
print("\nmask:", mask.values, "extracted ids:", ids.values)



## 15) Datetime & Time Series

- Convert: `pd.to_datetime`
- Components: `.dt.year`, `.dt.month`, `.dt.day`
- Resample: `.resample("M").sum()`, `.mean()`, etc. (needs datetime index)
- Rolling windows: `.rolling(window).mean()`


In [ ]:

# Build time series
dates = pd.date_range("2025-01-01", periods=90, freq="D")
visitors = (100 + np.cumsum(rng.normal(0, 2, size=len(dates)))).round().astype(int)
ts = pd.DataFrame({"visitors": visitors}, index=dates)

# Resample monthly
monthly = ts.resample("M").sum()

# Rolling mean (7-day)
roll7 = ts["visitors"].rolling(7, min_periods=1).mean()

print("Monthly sum:\n", monthly.head())

# Plot with matplotlib (single figure)
plt.figure()
plt.plot(ts.index, ts["visitors"], label="daily")
plt.plot(roll7.index, roll7, label="7-day mean")
plt.title("Daily Visitors with 7-Day Rolling Mean")
plt.xlabel("Date")
plt.ylabel("Visitors")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



## 16) Categorical Dtype

Use `category` for repeated strings → smaller memory & faster ops.


In [ ]:

df_cat = merged.copy()
df_cat["city"] = df_cat["city"].astype("category")
df_cat["region"] = df_cat["region"].astype("category")
print(df_cat.dtypes)
print("Memory usage before/after (bytes):",
      merged.memory_usage(deep=True).sum(), "→",
      df_cat.memory_usage(deep=True).sum())



## 17) `apply`, `map`, and `transform`

- Prefer vectorization first; then `map`/`replace` for simple dict-like mappings
- `apply` for row/column-wise Python functions (slower; use sparingly)
- `transform` for groupwise feature engineering that retains original length


In [ ]:

# map: replace values
region_map = {"North":"N", "South":"S"}
df_map = merged.copy()
df_map["region_short"] = df_map["region"].map(region_map)
print(df_map[["region","region_short"]].head())

# apply column-wise
def range_span(x):
    return x.max() - x.min()

span = merged[["temp","visitors"]].apply(range_span)
print("\nColumn spans:\n", span)

# transform with groupby (already shown z-score earlier)



## 18) Indexes & MultiIndex

- Set/reset index: `set_index`, `reset_index`
- MultiIndex: hierarchical row labels (e.g., city + month)
- Stack/unstack: reshape between rows and columns using index levels


In [ ]:

multi = (merged
         .assign(month_num = merged["month"].dt.month)
         .set_index(["city","month_num"])
         .sort_index())

print(multi.head())

# Unstack month to columns
unstacked = multi["visitors"].unstack("month_num")
print("\nUnstacked by month:\n", unstacked)

# Back to long
stacked = unstacked.stack()
print("\nStacked back (head):\n", stacked.head())



## 19) Plotting with pandas (Matplotlib backend)

We stick to Matplotlib explicitly for clarity—one chart per figure and no explicit colors.


In [ ]:

city_group = merged.groupby("city")["visitors"].mean()

plt.figure()
plt.bar(city_group.index, city_group.values)
plt.title("Average Visitors by City")
plt.xlabel("City")
plt.ylabel("Avg Visitors")
plt.show()



## 20) Display Options

- `pd.set_option("display.max_rows", 100)`
- `pd.set_option("display.precision", 3)`


In [ ]:

pd.set_option("display.max_rows", 20)
pd.set_option("display.precision", 3)
merged.head(10)



## 21) Performance Tips

- Prefer vectorized ops and built-ins (`assign`, `where`, `clip`, `min/max/mean`, etc.)
- Use appropriate dtypes (e.g., `category` for repeated strings)
- Avoid row-wise Python loops (`for` over DataFrame rows); use vectorization or `itertuples()`
- For large CSVs, use `dtype` and `usecols` in `read_csv` for speed and memory
- Use `groupby(..., observed=True)` with categoricals when appropriate



## 22) Exercises

1. From `merged`, compute total visitors per `region` per month. Which region had the highest total in March?
2. Create a new column `visitors_per_degree` = `visitors` / (`temp` + 10) and get its mean by `city`.
3. Build a pivot table with `index=city`, `columns=month (as month name)`, values=`visitors`, `aggfunc='sum'`.
4. Convert `region` to categorical and report memory saved vs object dtype.
5. Using the `ts` time series earlier, compute a 14-day rolling median and plot it.



## 23) Capstone Mini-Project 🏁

**Scenario:** You have city-level monthly data with temperature and visitors.

**Tasks:**
1. Create a **clean** DataFrame from scratch (or reuse `df_city`), ensuring `month` is datetime.
2. Add features:
   - `season` (Winter, Spring, Summer, Autumn) based on month
   - `is_peak` (visitors > city median)
3. Group by `city` and `season` to compute total visitors and average temperature.
4. Build a pivot table `index=city`, `columns=season`, values=total visitors.
5. Plot any **one** insightful chart and save it to disk.


In [ ]:

# 1) Start from df_city (already parsed dates)
data = df_city.copy()

# 2) Add season and is_peak
month = data["month"].dt.month
season_map = {12:"Winter", 1:"Winter", 2:"Winter",
              3:"Spring", 4:"Spring", 5:"Spring",
              6:"Summer", 7:"Summer", 8:"Summer",
              9:"Autumn", 10:"Autumn", 11:"Autumn"}
data["season"] = month.map(season_map)

city_median = data.groupby("city")["visitors"].transform("median")
data["is_peak"] = data["visitors"] > city_median

# 3) Group
grp = data.groupby(["city","season"]).agg(
    total_visitors=("visitors","sum"),
    avg_temp=("temp","mean"),
    count=("visitors","count")
).reset_index()

print(grp.head())

# 4) Pivot
pvt = grp.pivot(index="city", columns="season", values="total_visitors")
print("\nPivot visitors by season:\n", pvt)

# 5) Plot and save
plt.figure()
plt.bar(pvt.columns.astype(str), pvt.loc["Alpha"].values)
plt.title("Alpha: Total Visitors by Season")
plt.xlabel("Season")
plt.ylabel("Total Visitors")
out_path = "/mnt/data/alpha_visitors_by_season.png"
plt.savefig(out_path, dpi=300, bbox_inches="tight")
plt.show()

print("Saved chart to:", out_path)
